# Chargement des données de pandémies - Partie 1: Configuration MySQL

Ce notebook réalise la configuration de la connexion à la base de données MySQL et la préparation du schéma de la base de données.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et définition des paramètres de connexion à la base de données.

In [ ]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from pathlib import Path
import time

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

## 2. Configuration de la connexion à la base de données

Définition des paramètres de connexion à la base de données MySQL.

In [ ]:
# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

print("Paramètres de connexion à la base de données:")
for cle, valeur in config_db.items():
    if cle != "password":
        print(f"- {cle}: {valeur}")
    else:
        print(f"- {cle}: {'(aucun)' if valeur == '' else '******'}")

## 3. Fonction de connexion à la base de données

Définition d'une fonction pour établir une connexion à la base de données MySQL.

In [ ]:
def connecter_mysql(config=None, creer_db=False):
    """
    Établit une connexion à la base de données MySQL.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        creer_db (bool): Si True, crée la base de données si elle n'existe pas.
        
    Returns:
        tuple: (connexion, curseur) si la connexion est établie, (None, None) sinon.
    """
    if config is None:
        config = config_db.copy()
    
    try:
        # Si on veut créer la base de données, on se connecte sans spécifier la base de données
        if creer_db:
            config_sans_db = config.copy()
            database = config_sans_db.pop("database", None)
            connexion = mysql.connector.connect(**config_sans_db)
            curseur = connexion.cursor()
            
            # Création de la base de données si elle n'existe pas
            curseur.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
            curseur.execute(f"USE {database}")
            print(f"Base de données '{database}' créée ou sélectionnée avec succès.")
        else:
            # Connexion à la base de données existante
            connexion = mysql.connector.connect(**config)
            curseur = connexion.cursor()
            print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

## 4. Test de la connexion à la base de données

Test de la connexion à la base de données MySQL.

In [ ]:
# Test de la connexion à la base de données
print("Test de la connexion à la base de données...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql(creer_db=True)
    
    if connexion is not None and curseur is not None:
        # Vérification de la version de MySQL
        curseur.execute("SELECT VERSION()")
        version = curseur.fetchone()
        print(f"Version de MySQL: {version[0]}")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("Connexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors du test de la connexion à la base de données: {e}")

## 5. Définition du schéma de la base de données

Définition des requêtes SQL pour créer les tables de la base de données.

In [ ]:
# Définition des requêtes SQL pour créer les tables
requetes_creation_tables = {
    "calendrier": """
    CREATE TABLE IF NOT EXISTS calendrier (
        id_date INT PRIMARY KEY,
        date DATE NOT NULL,
        jour INT NOT NULL,
        mois INT NOT NULL,
        annee INT NOT NULL
    ) ENGINE=InnoDB;
    """,
    
    "localisation": """
    CREATE TABLE IF NOT EXISTS localisation (
        id_localisation INT PRIMARY KEY,
        pays VARCHAR(100) NOT NULL,
        continent VARCHAR(50)
    ) ENGINE=InnoDB;
    """,
    
    "pandemie": """
    CREATE TABLE IF NOT EXISTS pandemie (
        id_pandemie INT PRIMARY KEY,
        nom_pandemie VARCHAR(50) NOT NULL,
        description TEXT,
        date_debut DATE
    ) ENGINE=InnoDB;
    """,
    
    "donnees": """
    CREATE TABLE IF NOT EXISTS donnees (
        id_donnees INT PRIMARY KEY,
        id_date INT NOT NULL,
        id_localisation INT NOT NULL,
        id_pandemie INT NOT NULL,
        cas_confirmes INT NOT NULL,
        deces INT NOT NULL,
        FOREIGN KEY (id_date) REFERENCES calendrier(id_date),
        FOREIGN KEY (id_localisation) REFERENCES localisation(id_localisation),
        FOREIGN KEY (id_pandemie) REFERENCES pandemie(id_pandemie)
    ) ENGINE=InnoDB;
    """
}

# Affichage des requêtes SQL
print("Requêtes SQL pour créer les tables:")
for table, requete in requetes_creation_tables.items():
    print(f"\n--- Table {table} ---")
    print(requete)

## 6. Création des tables dans la base de données

Création des tables dans la base de données MySQL.

In [ ]:
# Création des tables dans la base de données
print("Création des tables dans la base de données...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql(creer_db=True)
    
    if connexion is not None and curseur is not None:
        # Désactivation temporaire des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        # Création des tables
        for table, requete in requetes_creation_tables.items():
            print(f"Création de la table {table}...")
            curseur.execute(requete)
            print(f"Table {table} créée avec succès.")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        
        # Validation des modifications
        connexion.commit()
        print("\nToutes les tables ont été créées avec succès.")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("Connexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la création des tables: {e}")

## 7. Vérification des tables créées

Vérification des tables créées dans la base de données MySQL.

In [ ]:
# Vérification des tables créées
print("Vérification des tables créées...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération de la liste des tables
        curseur.execute("SHOW TABLES;")
        tables = curseur.fetchall()
        
        print(f"Tables dans la base de données '{config_db['database']}':")
        for table in tables:
            print(f"- {table[0]}")
        
        # Vérification de la structure des tables
        for table in [table[0] for table in tables]:
            print(f"\nStructure de la table {table}:")
            curseur.execute(f"DESCRIBE {table};")
            colonnes = curseur.fetchall()
            for colonne in colonnes:
                print(f"- {colonne[0]}: {colonne[1]}")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des tables: {e}")

## Conclusion

Dans ce notebook, nous avons réalisé la configuration de la connexion à la base de données MySQL et la préparation du schéma de la base de données. Les principales opérations ont été :

1. Configuration des paramètres de connexion à la base de données
2. Test de la connexion à la base de données
3. Définition du schéma de la base de données
4. Création des tables dans la base de données
5. Vérification des tables créées

La base de données est maintenant prête à recevoir les données transformées. La prochaine étape consiste à charger les données dans la base de données, ce qui sera réalisé dans le prochain notebook.